In [65]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
from pyspark.sql.functions import *
import pandas as pd


In [66]:
#conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '40g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '1'), ('spark.cores.max', '1'), ('spark.driver.memory','50g')])

In [2]:
from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.types import *

In [3]:
weatherDF= spark.read.csv('/home/negah.moharrami@analytique.desjardins.com/project/CEBD1261/data/processed/New York_Weather_cyclical_taxi.csv', header=True, inferSchema=True)


In [4]:
numeric_features = [t[0] for t in weatherDF.dtypes if t[1] == 'int']
weatherDF.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
rides,8760,36.70616438356164,7.606905749665114,18,62
hour,8760,11.5,6.922581688234331,0,23
month,8760,6.526027397260274,3.448048134069238,1,12
day,8760,15.72054794520548,8.796749115212322,1,31
year,8760,2015.0,0.0,2015,2015


In [5]:
weatherDF.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: double (nullable = true)
 |-- rides: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- hour_sin: double (nullable = true)
 |-- hour_cos: double (nullable = true)
 |-- day_sin: double (nullable = true)
 |-- day_cos: double (nullable = true)
 |-- month_sin: double (nullable = true)
 |-- month_cos: double (nullable = true)



# Correlation

In [6]:
import pandas as pd
numeric_data = weatherDF.select('humidity','pressure','wind_speed', 'wind_direction', 'temperature').toPandas()
axs = pd.plotting.scatter_matrix(numeric_data, figsize=(8, 8));
n = 5
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

# Classification using logesticregression model 

In [7]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
numericCols = ['humidity', 'pressure', 'wind_speed', 'wind_direction', 'temperature']
stages=[]
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
stages += [assembler]


In [8]:
label_stringIdx = StringIndexer(inputCol = 'rides', outputCol = 'label')
stages += [label_stringIdx]

In [9]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(weatherDF)
weatherDF = pipelineModel.transform(weatherDF)
selectedCols = ['label', 'features']
weatherDF = weatherDF.select(selectedCols)
weatherDF.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [10]:
train, test = weatherDF.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 6111
Test Dataset Count: 2649


In [11]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [12]:
predictions = lrModel.transform(test)
predictions.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[39.0,1013.0,6.0,...|[5.95412141251474...|[0.02599687243441...|       4.0|
|  0.0|[39.0,1016.0,2.0,...|[6.05128577682234...|[0.03121381523653...|       4.0|
|  0.0|[47.0,1018.0,4.0,...|[6.28271153017067...|[0.03740690783165...|       4.0|
|  0.0|[48.0,1032.0,3.0,...|[6.63935356326205...|[0.04835149241832...|       1.0|
|  0.0|[49.0,1018.0,4.0,...|[6.31458079496595...|[0.03867957542700...|       1.0|
|  0.0|[51.0,1020.0,3.0,...|[6.38366345875886...|[0.04011805701094...|       1.0|
|  0.0|[52.0,1013.0,4.0,...|[6.38983662623400...|[0.04170284321494...|       1.0|
|  0.0|[52.0,1031.0,5.0,...|[6.71784348955568...|[0.05049672676987...|       1.0|
|  0.0|[53.0,1017.0,2.0,...|[6.59559060332573...|[0.04945270171799...|       1.0|
|  0.0|[53.0,101

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.3027364105095715


# Classification using Random forest

In [14]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictionsF = rfModel.transform(test)

predictionsF.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[39.0,1013.0,6.0,...|[1.13934467634065...|[0.05696723381703...|       4.0|
|  0.0|[39.0,1016.0,2.0,...|[0.88795305752630...|[0.04439765287631...|       8.0|
|  0.0|[47.0,1018.0,4.0,...|[1.08713658586893...|[0.05435682929344...|       7.0|
|  0.0|[48.0,1032.0,3.0,...|[0.95974154339131...|[0.04798707716956...|       8.0|
|  0.0|[49.0,1018.0,4.0,...|[1.00457160125470...|[0.05022858006273...|       5.0|
|  0.0|[51.0,1020.0,3.0,...|[2.19913302564692...|[0.10995665128234...|       4.0|
|  0.0|[52.0,1013.0,4.0,...|[2.46112855782399...|[0.12305642789119...|       0.0|
|  0.0|[52.0,1031.0,5.0,...|[2.06801941869780...|[0.10340097093489...|       5.0|
|  0.0|[53.0,1017.0,2.0,...|[2.34450221207990...|[0.11722511060399...|       0.0|
|  0.0|[53.0,101

In [15]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictionsF, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.14804339677838108


In [19]:
predictions.write.save('/home/negah.moharrami@analytique.desjardins.com/project/CEBD1261/data/prediction_taxi.csv',header=True, sep=';')